In [2]:
#Importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. Configurações Iniciais

In [3]:
arquivos = {
    'IDLE_BRUTO.csv': 0,
    'GAMING_BRUTO.csv': 1,
    'OBSTRUCAO_BRUTO.csv': 2
}
colunas = ['timestamp', 'temp', 'umidade', 'label_str']

# 2. Carregamento e Rotulagem

In [4]:
dfs = []
for nome_arq, classe in arquivos.items():
    try:
        # Carrega forçando os nomes das colunas (já que os arquivos brutos não têm header)
        df_temp = pd.read_csv(nome_arq, names=colunas, header=None)
        df_temp['classe'] = classe
        dfs.append(df_temp)
        print(f"Lido: {nome_arq} | Amostras: {len(df_temp)}")
    except FileNotFoundError:
        print(f"ERRO: Arquivo {nome_arq} não encontrado!")
# Concatenar todos
df_completo = pd.concat(dfs, ignore_index=True)

Lido: IDLE_BRUTO.csv | Amostras: 2433
Lido: GAMING_BRUTO.csv | Amostras: 3382
Lido: OBSTRUCAO_BRUTO.csv | Amostras: 3766


# 3. Balanceamento (Down-sampling)

In [5]:
menor_classe = df_completo['classe'].value_counts().min()
df_balanceado = df_completo.groupby('classe').sample(n=menor_classe, random_state=42)

print(f"\nDataset balanceado para {menor_classe} amostras por classe.")


Dataset balanceado para 2433 amostras por classe.


# 4. Extração de Características e Normalização

In [6]:
X = df_balanceado[['temp', 'umidade']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. GERADOR DE CONSTANTES PARA O C (BITDOGLAB)

In [7]:
mean = scaler.mean_
std = np.sqrt(scaler.var_)

print("\n" + "="*45)
print("   COPIE E COLE NO SEU CÓDIGO C (.c)   ")
print("="*45)
print(f"const float MEAN_TEMP = {mean[0]:.8f}f;")
print(f"const float STD_TEMP  = {std[0]:.8f}f;")
print(f"const float MEAN_UMID = {mean[1]:.8f}f;")
print(f"const float STD_UMID  = {std[1]:.8f}f;")
print("="*45)


   COPIE E COLE NO SEU CÓDIGO C (.c)   
const float MEAN_TEMP = 47.52881080f;
const float STD_TEMP  = 4.63896743f;
const float MEAN_UMID = 37.43903685f;
const float STD_UMID  = 7.40280150f;


# 6. Salvar para o treino

In [8]:
df_balanceado.to_csv('dataset_pronto_treino.csv', index=False)
print("\nArquivo 'dataset_pronto_treino.csv' gerado com sucesso!")


Arquivo 'dataset_pronto_treino.csv' gerado com sucesso!
